In [21]:
import warnings
warnings.filterwarnings('ignore')

In [22]:
import pandas as pd

#my helper functions from ipynb file
from ipynb.fs.full.PostgreHelper import * 

# Extract

In [27]:
#my sample google sheet
sheet_id = '1xo1_DZEVBiVz39S5Ln3CSnagbPE78cvBDKtwqtC3hKI'

#extract all tables from the sheet
student_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet=Students"
df_student = pd.read_csv(student_url)

st_course_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet=Student_Courses"
df_st_course = pd.read_csv(st_course_url)

course_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet=Courses_Info"
df_course_info = pd.read_csv(course_url)

payment_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet=Payment_Method"
df_payment = pd.read_csv(payment_url)

purchased_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet=Purchased_Info"
df_purchase = pd.read_csv(purchased_url)

In [28]:
df_st_course

,student_id,course_id,status,completed date,feedback
0,S001,COURSE001,Completed,2/3/2022,feedback001
1,S002,COURSE002,Completed,2/1/2022,feedback002
2,S003,COURSE003,In progress,NaN,feedback003
3,S003,COURSE004,In progress,NaN,feedback004
4,S004,COURSE004,Completed,2/3/2022,feedback005
5,S005,COURSE005,In progress,NaN,feedback006
6,S006,COURSE006,Completed,2/5/2022,feedback007
7,S007,COURSE007,In progress,NaN,feedback008
8,S007,COURSE008,In progress,NaN,feedback009
9,S008,COURSE008,Completed,2/7/2022,feedback010


# Transform (Dimensional Model)

# Load

In [ ]:
#connect to my Postgre database that hosted on ElephantSql with free instance
connection, cursor = connect(host = 'arjuna.db.elephantsql.com', 
                             database = '*******', 
                             user = '*******', 
                             password = '*******', 
                             port = '5432')

In [ ]:
#create new table for the above youtube data
query = 'CREATE TABLE youtube_channel \
        (id SERIAL PRIMARY KEY, title text, url text, liked int, disliked int, views int, comment_count int); '
create_table(connection, cursor, query)

In [ ]:
#inserting the dataframe into youtube_channel table

#convert dataframe in tuples to iterate
tuples = [tuple(x) for x in df.to_numpy()]

#df columns into string
cols = ','.join(list(df.columns))

#insert query
query = "INSERT INTO youtube_channel(%s) VALUES(%%s,%%s,%%s,%%s,%%s,%%s)" % (cols)

#inserting with executemany function
try:
    cursor.executemany(query, tuples)
    print("Data inserted using execute_many() successfully!")
except (Exception, psycopg2.DatabaseError) as err:
    print(err)
    cursor.close()

In [ ]:
#read PostgreSQL table into pandas dataframe
data = pd.read_sql_query(con = connection, sql = 'SELECT * FROM youtube_channel')
data.head(10)